In [ ]:
import os
import random
import subprocess
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from datetime import datetime

random.seed(42)

# ================== CONFIGURAÇÃO DA CALIBRAÇÃO DE INÓCULO ==================
BIOLOGICAL_PARAMS_PATH = r"C:\Users\Gustavo\OneDrive\Área de Trabalho\mestrado\pesquisa\tables\parametros_biologicos_sul.xlsx"
ID_CURVA_PARA_AJUSTE = 4
EXPERIMENT_FILE_AJUSTE = "LOND0504.SBX"

# --- Caminhos (geralmente não mudam) ---
OBSERVED_DATA_PATH = r"C:\Users\Gustavo\OneDrive\Área de Trabalho\mestrado\pesquisa\dados\CURVE_DATABASE_Severity&Defol.xlsx"
OBSERVED_SHEET_NAME = "organizados"
PARAMS_FILE_DSSAT = r"C:\DSSAT48\Soybean\disease_parameters.txt"
DSSAT_EXE = r"C:\dssat-csm-os-develop\build\bin\Debug\dscsm048.exe"
EXP_DIR = r"C:\DSSAT48\Soybean"
DSSAT_OUTPUT_FILE = os.path.join(EXP_DIR, "DISEASE_DEVELOPMENT.OUT")

# --- PASTA DE SAÍDA PARA OS RESULTADOS ---
OUTPUT_DIR = r"C:\Users\Gustavo\OneDrive\Área de Trabalho\mestrado\pesquisa\output"

# ================== FUNÇÕES AUXILIARES ==================

def generate_random_inoculum(base_params):
    final_params = base_params.copy()
    inoculum_ranges = {'NDS': (1000, 100000), 'DAE_START': (20, 40)}
    final_params['NDS'] = random.randint(*inoculum_ranges['NDS'])
    final_params['DAE_START'] = random.randint(*inoculum_ranges['DAE_START'])
    return final_params

def write_params_to_file(params, header_lines, disease_name, sim_number=1):
    new_id = f"D{sim_number:05d}"
    line = (
        f"{new_id:<7s} {disease_name:<17s}"
        f"{int(params['NDS']):>10d}{params['LESION_S']:>16.6f}"
        f"{int(params['KVERHULST']):>16d}{params['RVERHULST']:>16.3f}"
        f"{params['YMAX']:>16.3f}{params['COF_A']:>16.3f}{params['COF_B']:>16.3f}"
        f"{params['TMIN_G']:>16.3f}{params['TOT_G']:>16.3f}{params['TMAX_G']:>16.3f}"
        f"{params['TMIN_D']:>16.3f}{params['TOT_D']:>16.3f}{params['TMAX_D']:>16.3f}"
        f"{params['LDMin']:>16.3f}{params['LESIONAGEOPT']:>16.3f}{params['LESLIFEMAX']:>16.3f}"
        f"{int(params['DAE_START']):>16d}"
    )
    with open(PARAMS_FILE_DSSAT, 'w', encoding='utf-8') as f:
        f.writelines(header_lines)
        f.write(line + '\n')

def run_dssat_simulation(experiment_name):
    try:
        subprocess.run([DSSAT_EXE, "A", experiment_name], cwd=EXP_DIR,
                       check=True, capture_output=True, text=True, encoding='latin1')
        return True
    except (subprocess.CalledProcessError, FileNotFoundError):
        return False

# ---------- Leitura ROBUSTA do OUT (pega apenas o último RUN) ----------
def read_disease_out_last_run(path_out: str) -> pd.DataFrame:
    with open(path_out, 'r', encoding='latin1', errors='ignore') as f:
        lines = f.readlines()

    header_idx = None
    # aceitamos ambos cabeçalhos (com/sem LA_INFECT e NEW_LOSS)
    must_have = ["RUN", "YYDOY", "DAS", "LAI_HEALTH", "LA_DISEASE", "LAI_TOTAL"]
    for i, ln in enumerate(lines):
        if all(k in ln for k in must_have):
            header_idx = i
            break
    if header_idx is None:
        raise ValueError("Cabeçalho não encontrado em DISEASE_DEVELOPMENT.OUT")

    # nomes na ordem do WRITE do Fortran mais novo
    colnames = ["RUN","YYDOY","DAS","LAI_HEALTH","LA_DISEASE","LA_INFECT","NEW_LOSS",
                "LWD","RH","FT","LAI_TOTAL","SUM7","NSPRAYS","FUNG_ACT"]
    df = pd.read_fwf(path_out, skiprows=header_idx+1, names=colnames, engine="python", encoding='latin1')

    # Conversões numéricas essenciais
    for c in ["RUN","DAS","LAI_HEALTH","LA_DISEASE","LA_INFECT","NEW_LOSS","LAI_TOTAL"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors='coerce')

    df = df.dropna(subset=["RUN","DAS"])
    if df.empty:
        raise ValueError("OUT sem linhas úteis após o cabeçalho.")

    last_run = int(df["RUN"].max())
    df = df[df["RUN"] == last_run].copy()
    df["DAS"] = df["DAS"].astype(int)
    df.sort_values("DAS", inplace=True)

    # garante colunas-chave
    for k in ["LAI_HEALTH","LA_DISEASE","LA_INFECT","LAI_TOTAL"]:
        if k not in df.columns:
            df[k] = np.nan

    return df[["DAS","LAI_HEALTH","LA_DISEASE","LA_INFECT","LAI_TOTAL"]]

# ---------- Severidade MONÓTONA (100 * LA_DISEASE / LAI_PEAK) ----------
def calculate_severity(df_last_run: pd.DataFrame, last_obs_day: int = None) -> pd.DataFrame:
    df = df_last_run.copy()
    # saneamento
    for c in ["DAS","LAI_HEALTH","LA_DISEASE","LAI_TOTAL"]:
        df[c] = pd.to_numeric(df[c], errors='coerce')
    df = df.dropna(subset=["DAS","LAI_TOTAL","LA_DISEASE"]).sort_values("DAS")
    if df.empty:
        return pd.DataFrame(columns=["DAS","Severity(%)"])

    df["DAS"] = df["DAS"].astype(int)

    # define denominador fixo (LAI_PEAK): pico até o último dia observado (se dado)
    if last_obs_day is not None:
        lai_peak = float(df.loc[df["DAS"] <= last_obs_day, "LAI_TOTAL"].max())
        if not np.isfinite(lai_peak) or lai_peak <= 0:
            lai_peak = float(df["LAI_TOTAL"].max())
    else:
        lai_peak = float(df["LAI_TOTAL"].max())

    lai_peak = max(lai_peak, 1e-9)

    # severidade cumulativa monotônica
    sev = 100.0 * (df["LA_DISEASE"].clip(lower=0.0)) / lai_peak
    sev = np.clip(sev.values, 0.0, 100.0)
    # blindagem contra pequenos “dentes”
    sev = np.maximum.accumulate(sev)

    out = pd.DataFrame({"DAS": df["DAS"].values, "Severity(%)": sev})
    return out

def comparar_pontos_observados(df_sim_completo, df_obs_geral, id_curva_selecionada):
    df_obs_curva = df_obs_geral[df_obs_geral['Curva'] == id_curva_selecionada].copy()
    if df_obs_curva.empty: return None, np.inf, -np.inf, None

    df_obs_curva['Av_DAS'] = pd.to_numeric(df_obs_curva['Av_DAS'], errors='coerce')
    df_obs_curva.dropna(subset=['Av_DAS', 'Severidade (%)'], inplace=True)
    if len(df_obs_curva) < 3: return None, np.inf, -np.inf, None

    ultimo_dia_observado = int(df_obs_curva['Av_DAS'].max())
    df_sim_filtrado = df_sim_completo[df_sim_completo['DAS'] <= ultimo_dia_observado].copy()

    df_obs_curva.rename(columns={'Av_DAS': 'DAS', 'Severidade (%)': 'Severity(%)_obs'}, inplace=True)
    df_obs_filtrado_para_plot = df_obs_curva[['DAS', 'Severity(%)_obs']].drop_duplicates(subset=['DAS'])

    merged = pd.merge(df_sim_filtrado, df_obs_filtrado_para_plot, on='DAS', how='inner').rename(
        columns={'Severity(%)': 'Severity(%)_sim'})
    if len(merged) < 3: return None, np.inf, -np.inf, None
    
    rmse = mean_squared_error(merged['Severity(%)_obs'], merged['Severity(%)_sim'], squared=False)
    r2 = r2_score(merged['Severity(%)_obs'], merged['Severity(%)_sim'])
    return merged, rmse, r2, df_sim_filtrado

# ================== FUNÇÃO DE PLOTAGEM (inalterada) ==================

def plotar_resultados(best_sim_df, df_comparativo, best_params, data_plantio):
    id_curva = best_params['ID_Curva_Obs']
    TITLE_FONTSIZE, TICK_LABEL_FONTSIZE, STATS_FONTSIZE = 23.5, 22.5, 22.5
    
    plt.style.use('seaborn-v0_8-paper')
    fig, ax = plt.subplots(figsize=(9, 7))
    
    ax.scatter(df_comparativo['DAS'], df_comparativo['Severity(%)_obs'], label='Observado', color='crimson', s=80, zorder=5, edgecolors='black')
    ax.plot(best_sim_df['DAS'], best_sim_df['Severity(%)'], label='Simulado', color='black', linewidth=2.5)
    
    props = dict(boxstyle='round,pad=0.5', facecolor='white', alpha=0.85)

    stats_text = (f"RMSE = {best_params['RMSE']:.2f}\n" f"$R^2$ = {best_params['R2']:.3f}")
    ax.text(0.05, 0.95, stats_text, transform=ax.transAxes, fontsize=STATS_FONTSIZE, verticalalignment='top', bbox=props)

    inoculum_text = (f"NDS = {int(best_params['NDS'])} (spores m$^{{-2}}$)\n" f"DAE_START = {int(best_params['DAE_START'])}")
    ax.text(0.05, 0.75, inoculum_text, transform=ax.transAxes, fontsize=STATS_FONTSIZE, verticalalignment='top', bbox=props)
            
    titulo_grafico = f"Londrina - PR | Sowing: {data_plantio.strftime('%d/%m/%Y')}"
    ax.set_title(titulo_grafico, fontsize=TITLE_FONTSIZE, pad=20)
    
    ax.tick_params(axis='both', which='major', labelsize=TICK_LABEL_FONTSIZE)
    ax.set_ylim(0, 100)
    plt.tight_layout()
    
    nome_arquivo_grafico = f"grafico_ajuste_final_curva_{id_curva}.png"
    caminho_completo_grafico = os.path.join(OUTPUT_DIR, nome_arquivo_grafico)
    plt.savefig(caminho_completo_grafico, dpi=300)
    print(f"Gráfico salvo em: {caminho_completo_grafico}")

    plt.show()

# ================== EXECUÇÃO ==================
def main(id_curva, experiment_file, n_sim=200):
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    try:
        base_params_df = pd.read_excel(BIOLOGICAL_PARAMS_PATH)
        base_biological_params = base_params_df.to_dict('records')[0]
        
        df_observado_geral = pd.read_excel(OBSERVED_DATA_PATH, sheet_name=OBSERVED_SHEET_NAME)
        info_curva = df_observado_geral[df_observado_geral['Curva'] == id_curva].iloc[0]
        data_plantio = pd.to_datetime(info_curva['Data_Plantio'])
        ultimo_obs = int(pd.to_numeric(
            df_observado_geral.loc[df_observado_geral['Curva'] == id_curva, 'Av_DAS'],
            errors='coerce'
        ).max())

        with open(PARAMS_FILE_DSSAT, 'r', encoding='utf-8') as f:
            lines = f.readlines()
            var_index = next(i for i, l in enumerate(lines) if l.strip().startswith('@VAR#'))
            header = lines[:var_index + 3]
            disease_name = lines[var_index + 3].split()[1]
            
    except (FileNotFoundError, IndexError) as e:
        print(f"Erro ao carregar arquivos de configuração: {e}")
        return
    except KeyError as e:
        print(f"Erro: A coluna {e} não foi encontrada na sua planilha. Verifique se a coluna 'Data_Plantio' existe.")
        return

    best_rmse = np.inf
    best_params_final = None
    best_sim_df_filtrado = None
    melhor_comparativo_df = None

    print(f"--- Iniciando ajuste de inóculo para a Curva {id_curva} ---")
    for sim_idx in range(n_sim):
        params_completos = generate_random_inoculum(base_biological_params)
        write_params_to_file(params_completos, header, disease_name, sim_idx + 1)
        
        if not run_dssat_simulation(experiment_file):
            continue
        if os.path.exists(DSSAT_OUTPUT_FILE):
            try:
                # leitura robusta + severidade monótona até o último dia observado
                df_out = read_disease_out_last_run(DSSAT_OUTPUT_FILE)
                df_sim_calculated = calculate_severity(df_out, last_obs_day=ultimo_obs)

                comparativo_atual_df, rmse_atual, r2_atual, sim_df_filtrado_atual = comparar_pontos_observados(
                    df_sim_calculated, df_observado_geral, id_curva
                )

                if comparativo_atual_df is not None and rmse_atual < best_rmse:
                    best_rmse = rmse_atual
                    best_params_final = params_completos.copy()
                    best_params_final.update({
                        'RMSE': rmse_atual, 'R2': r2_atual,
                        'ID_Curva_Obs': id_curva, 'Experiment': experiment_file
                    })
                    best_sim_df_filtrado = sim_df_filtrado_atual.copy()
                    melhor_comparativo_df = comparativo_atual_df.copy()
                    print(f"  Sim {sim_idx+1}/{n_sim} | Novo melhor RMSE: {best_rmse:.4f}, R2: {r2_atual:.4f}")
            except Exception:
                continue

    if best_sim_df_filtrado is not None:
        print(f"\nMelhor ajuste de inóculo para Curva {id_curva} encontrado.")
        plotar_resultados(best_sim_df_filtrado, melhor_comparativo_df, best_params_final, data_plantio)
        
        output_filename = f"parametros_finais_curva_{id_curva}.xlsx"
        caminho_completo_excel = os.path.join(OUTPUT_DIR, output_filename)
        pd.DataFrame([best_params_final]).to_excel(caminho_completo_excel, index=False)
        print(f"Parâmetros finais exportados para: {caminho_completo_excel}")
    else:
        print(f"\nNenhuma simulação válida foi encontrada para a Curva {id_curva} após {n_sim} tentativas.")

# ================== INICIAR ==================
if __name__ == "__main__":
    main(id_curva=ID_CURVA_PARA_AJUSTE, experiment_file=EXPERIMENT_FILE_AJUSTE, n_sim=300)

--- Iniciando ajuste de inóculo para a Curva 4 ---
  Sim 1/300 | Novo melhor RMSE: 14.8449, R2: 0.7445
  Sim 5/300 | Novo melhor RMSE: 13.1463, R2: 0.7997
  Sim 40/300 | Novo melhor RMSE: 13.0873, R2: 0.8015


KeyboardInterrupt: 